<a href="https://colab.research.google.com/github/VCHULANI15/pricing-options-with-machine-learning/blob/main/Pricing_Options_With_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing necessary libraries for manipulating data
import numpy as np
import pandas as pd
import seaborn as sns

#Plot Library
import matplotlib.pyplot as plt
import plotly.express as px
from plotnine import *

#Sklearn Library
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, median_absolute_error, mean_squared_error, r2_score, mean_absolute_error, classification_report
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

#Torch Data
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import norm
from itertools import product
